# Weekly project 6
Today we will continue work from monday.
We will follow the style of last week.

Weekly project:
- You will need to implement your own k-means algorithm. (So you are not allowed to use the one in *sklearn*
- it should be able to cluster each of the different figures.
- Extend your k-means so it finds the optimal amount of clusters.
Challenge
- Implement the mean shift clustering algorithm


In [71]:
import numpy as np
import open3d as o3d
import copy
import matplotlib.pyplot as plt
#%matplotlib notebook

def draw_labels_on_model(pcl, labels):
    cmap = plt.get_cmap("tab20")
    pcl_temp = copy.deepcopy(pcl)
    max_label = labels.max()
    colors = cmap(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    pcl_temp.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_geometries([pcl_temp])

d = 4
mesh = o3d.geometry.TriangleMesh.create_tetrahedron().translate((-d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_octahedron().translate((0, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_icosahedron().translate((d, 0, 0))
mesh += o3d.geometry.TriangleMesh.create_torus().translate((-d, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_mobius(twists=1).translate((0, -d, 0))
mesh += o3d.geometry.TriangleMesh.create_mobius(twists=2).translate((d, -d, 0))

## apply k means on this
point_cloud_ez = mesh.sample_points_uniformly(int(1e3)) # Ez mode
point_cloud_hard = mesh.sample_points_uniformly(int(1e5), 0.5) # Hard mode

In [73]:
o3d.visualization.draw_geometries([point_cloud_ez])
o3d.visualization.draw_geometries([point_cloud_hard])

In [21]:
points = np.asarray(point_cloud_ez.points)
points.shape

(1000, 3)

In [123]:
def k_mean(n_clusters, points, threshold=0.002, restarts=1):
    '''
    K -means, yeey.
    '''
    best_silhouette_score = 0
    for r in range(restarts):
        print("Restart no.", r, end="\t")
        # Step 1 choose random clusters
        centeroids_i = np.random.randint(0, len(points), n_clusters)
        centroids = points[centeroids_i]
        biggest_change = threshold + 1
        iterations = 0

        while biggest_change > threshold:
            assignments = np.zeros(shape=len(points))
            # Step 2 Assagn each point to closest centroid
            # TODO: KD trees
            for i, p in enumerate(points):
                closest_centroid = 0
                closest_dist = float("inf")
                for j, c in enumerate(centroids):
                    dist = euclidean_dist(p, c) # This is the time killer
                    if dist < closest_dist:
                        closest_centroid = j
                        closest_dist = dist
                assignments[i] = closest_centroid
            
            # Step 3 re-calculate the mean of each cluster
            biggest_change = 0
            for i in range(n_clusters):
                old_centroid = centroids[i].copy()
                centroids[i] = np.mean(points[np.where(assignments==i)[0]], axis=0)
                change = euclidean_dist(old_centroid, centroids[i])
                if change > biggest_change:
                    biggest_change = change
            
            # Step 4 - repeat!
            iterations += 1
            

        # Step 5: calculate solouette score, random restart
        silhouette_score = float("inf")
        for i in range(n_clusters):
            for j in range(n_clusters):
                if i!=j:
                    dist = euclidean_dist(centroids[i], centroids[j])
                    if dist < silhouette_score:
                        silhouette_score = dist

        if silhouette_score > best_silhouette_score:
            print("New best!", end="")
            best_assignments = assignments.copy()
            best_centroids = centroids.copy()
            best_iterations = iterations
            best_silhouette_score = silhouette_score
        print("")

    return best_assignments, best_centroids, best_iterations


def euclidean_dist(point1, point2):
    return np.linalg.norm((point2-point1))

In [121]:
assignments, centroids, iterations = k_mean(6, points, threshold=0.002, restarts=15)

Restart no. 0	New best!
Restart no. 1	
Restart no. 2	New best!
Restart no. 3	
Restart no. 4	
Restart no. 5	
Restart no. 6	
Restart no. 7	
Restart no. 8	
Restart no. 9	
Restart no. 10	
Restart no. 11	
Restart no. 12	
Restart no. 13	
Restart no. 14	


In [122]:
draw_labels_on_model(point_cloud_ez, assignments)

**TODO: Speed up the algortihm**
- Instead of recomputing the distances each restart of k-means, seperate them in some tree structure (KD trees)
- 